# Test zum speichern der Features

In [1]:
import torch

In [2]:
torch.random.manual_seed(1)

In [3]:
from glob import glob
import os

In [4]:
import os
from pathlib import Path
import pandas as pd

cwd = Path.cwd()
DATA_DIR = os.path.join(cwd, 'data')
FEATURES_DIR = os.path.join(DATA_DIR, 'features')

gs_df = pd.read_csv(os.path.join(DATA_DIR, 'gs.csv'))

,ID,video_id,segment,start,end,humor,coach,partition
0,baum_01_1116500_1133500_0_2000,baum_01,baum_01_1116500_1133500,0,2000,0.0,baum,train
1,baum_01_1116500_1133500_1000_3000,baum_01,baum_01_1116500_1133500,1000,3000,0.0,baum,train
2,baum_01_1116500_1133500_2000_4000,baum_01,baum_01_1116500_1133500,2000,4000,0.0,baum,train
3,baum_01_1116500_1133500_3000_5000,baum_01,baum_01_1116500_1133500,3000,5000,0.0,baum,train
4,baum_01_1116500_1133500_4000_6000,baum_01,baum_01_1116500_1133500,4000,6000,0.0,baum,train


In [19]:
expected_sr = 16000

In [1]:
import librosa
from numpy.lib.stride_tricks import sliding_window_view
from scipy.signal import resample
import numpy as np
expected_sr = 16000

def load_windowed_signal(file, window=3, step=.5):
    signal, sr = librosa.load(file)
    intermediate = int(signal.__len__() * (expected_sr / sr))
    signal = resample(signal,intermediate)
    windows = sliding_window_view(signal,window * expected_sr)
    windows = windows[::int(expected_sr * step)]
    remaining = signal.__len__() % int(expected_sr * step) + int(window * expected_sr) - int(step * expected_sr)
    while remaining > expected_sr:
        windows = np.concatenate((windows,[np.pad(signal[-remaining:],(0,window * expected_sr - remaining),'constant',constant_values=(0,0))]))
        remaining -= int(step * expected_sr)
    return windows


In [2]:
import numpy as np
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)


class RegressionHead(nn.Module):
    r"""Classification head."""

    def __init__(self, config):

        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):

        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)

        return x


class EmotionModel(Wav2Vec2PreTrainedModel):
    r"""Speech emotion classifier."""

    def __init__(self, config):

        super().__init__(config)

        self.config = config
        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = RegressionHead(config)
        self.init_weights()

    def forward(
            self,
            input_values,
    ):

        outputs = self.wav2vec2(input_values)
        hidden_states = outputs[0]
        hidden_states = torch.mean(hidden_states, dim=1)
        logits = self.classifier(hidden_states)

        return hidden_states, logits



# load model from hub
device = 'cuda'
model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = EmotionModel.from_pretrained(model_name).to(device)

# dummy signal
sampling_rate = 16000
signal = np.zeros((1, sampling_rate), dtype=np.float32)


def process_func(
    x: np.ndarray,
    sampling_rate: int,
    embeddings: bool = False,
) -> np.ndarray:
    r"""Predict emotions or extract embeddings from raw audio signal."""

    # run through processor to normalize signal
    # always returns a batch, so we just get the first entry
    # then we put it on the device
    y = processor(x, sampling_rate=sampling_rate)
    y = y['input_values']
    y = np.array(y)
    #y = y.reshape(1, -1)
    y = torch.from_numpy(y).to(device)
    # run through model
    with torch.no_grad():
        y = model(y)[0 if embeddings else 1]

    # convert to numpy
    y = y.detach().cpu().numpy()

    return y


#print(process_func(signal, sampling_rate))
#  Arousal    dominance valence
# [[0.5460754  0.6062266  0.40431657]]

#print(process_func(signal, sampling_rate, embeddings=True))

c:\Users\Fabio\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
torch.cuda.empty_cache



<function torch.cuda.memory.empty_cache() -> None>

In [18]:
# load features:
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import math

i = -1
cwd = Path.cwd()
DATA_DIR = os.path.join(cwd, 'data')
gs_df = pd.read_csv(os.path.join(DATA_DIR, 'gs.csv'))

# single load of coaches, because the programm crashed and loading old coaches again would have aken too much time
gs_df = gs_df[gs_df.coach=='breitenreiter']
last_segment = ""
print(gs_df.shape)
len_df = gs_df.shape[0]
SOUND_DIR = os.path.join(cwd, 'audio')
for _, row in gs_df.iterrows():
    i += 1
    if last_segment == row["segment"]:
        continue
    print(str((i*100)/len_df) + "%")
    last_segment = row["segment"]
    file_sound = f'{SOUND_DIR}/{row["coach"]}/{row["video_id"]}/{row["segment"]}.wav'
    sound_array = load_windowed_signal(file_sound, window=3, step=.5)
    save = []
    i_inner = 0
    
    # pushung chunks of data to the Grafics card
    array_part_size = 90
    for i_in in range(math.ceil(sound_array.shape[0]/array_part_size)):
        part_sound = sound_array[array_part_size*i_in:array_part_size*(i_in+1)]
        part_process = process_func(part_sound, expected_sr, embeddings=True)
        if save.__len__() == 0:
            save = part_process
        else:
            save = np.append(part_process, save, axis=0)
    
    timestamps = list(range(0, sound_array.__len__()*500, 500))
    dic = {'timestamps': timestamps}
    save = save.transpose()
    i_inner = 1
    for embed in save:
        dic[str(i_inner)] = embed
        i_inner += 1     
    df_features = pd.DataFrame(dic)
    df_features.to_csv(f'{SOUND_DIR}/features/{row["coach"]}/{row["segment"]}.csv', index=False)





(2711, 8)
0.0%
2.3607524898561416%
4.352637403172261%
6.565842862412394%
9.221689413500552%
11.287347842124678%
13.463666543710808%
16.045739579490963%
17.078568793803026%
19.402434526005162%
20.582810770933236%
22.50092216894135%
24.52969383991147%
25.710070084839543%
26.6691257838436%
27.443747694577645%
28.255256362965696%
30.13648100331981%
33.75138325341202%
36.00147547030616%
37.47694577646625%
37.58760604942825%
38.952416082626335%
41.57137587606049%
44.374769457764664%
45.186278126152715%
49.61268904463298%
50.719291774253044%
51.6783474732571%
53.3382515676872%
55.99409811877536%
58.09664330505348%
60.16230173367761%
61.08447067502767%
62.04352637403172%
62.744374769457764%
63.77720398376983%
64.81003319808188%
68.68314275175211%
70.74880118037625%
72.55625230542235%
74.17926964219845%
77.16709701217263%
80.22869789745481%
85.39284396901512%
87.67982294356327%
90.33566949465143%
93.76613795647363%
97.71302102545187%


In [10]:
# just to see the coaches
gs_df = pd.read_csv(os.path.join(DATA_DIR, 'gs.csv'))
gs_df['coach'].unique()


array(['baum', 'breitenreiter', 'hasenhuttl', 'hecking', 'herrlich',
       'kovac', 'nagelsmann', 'schwarz', 'streich', 'tedesco'],
      dtype=object)